In [ ]:
###############################3######################## Color Tracking Code ##############################################################
import time
import numpy as np
import cv2

# Open the default camera (camera index 0)
cap = cv2.VideoCapture(0)

# Function to do nothing (used for creating trackbars)
def nothing(x):
    pass

# Create a window named "Trackbars" to adjust HSV values
cv2.namedWindow("Trackbars")

# Create trackbars to adjust lower and upper HSV values
cv2.createTrackbar("L - H", "Trackbars", 0, 179, nothing)
cv2.createTrackbar("L - S", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("L - V", "Trackbars", 0, 255, nothing)
cv2.createTrackbar("U - H", "Trackbars", 179, 179, nothing)
cv2.createTrackbar("U - S", "Trackbars", 255, 255, nothing)
cv2.createTrackbar("U - V", "Trackbars", 255, 255, nothing)

# Main loop
while True:
    # Read a frame from the camera
    ret, frame = cap.read()

    # Resize the frame to (640, 480)
    frame = cv2.resize(frame, (640, 480))

    # Convert the frame to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Get the current positions of the trackbars
    l_h = cv2.getTrackbarPos("L - H", "Trackbars")
    l_s = cv2.getTrackbarPos("L - S", "Trackbars")
    l_v = cv2.getTrackbarPos("L - V", "Trackbars")
    u_h = cv2.getTrackbarPos("U - H", "Trackbars")
    u_s = cv2.getTrackbarPos("U - S", "Trackbars")
    u_v = cv2.getTrackbarPos("U - V", "Trackbars")

    # Create lower and upper HSV ranges based on trackbar values
    lower_range = np.array([l_h, l_s, l_v])
    upper_range = np.array([u_h, u_s, u_v])

    # Create a binary mask using the inRange function
    mask = cv2.inRange(hsv, lower_range, upper_range)

    # Apply the mask to the original frame using bitwise_and
    result = cv2.bitwise_and(frame, frame, mask=mask)

    # Show the thresholded mask and the result
    cv2.imshow("mask", mask)
    cv2.imshow("result", result)

    # Check for the 'q' key to exit the loop
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

In [ ]:
####################################################### Final Distance Detect Code ################################################################
import cv2
import numpy as np

# Open the default camera (camera index 0)
cap = cv2.VideoCapture(0)

# Set the initial lower and upper HSV range for the known object
lower_range = np.array([65, 10, 77])
upper_range = np.array([86, 82, 225])

# Define known parameters for distance calculation
Known_distance = 40.0
Known_width = 18.0

# Variables to store detected object information
Detected_object = 0
Object_distance = 0

# Function to calculate the focal length of the camera
def Focal_Length_Finder(Known_distance, real_width, width_in_rf_image):
    focal_length = (width_in_rf_image * Known_distance) / real_width
    return focal_length

# Function to extract object width from an image
def obj_data(img):
    obj_width = 0
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_range, upper_range)
    _, mask1 = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
    cnts, _ = cv2.findContours(mask1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for c in cnts:
        x = 600
        if cv2.contourArea(c) > x:
            x, y, w, h = cv2.boundingRect(c)
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            obj_width = w
            return obj_width
    return 0

# Function to calculate the distance to the object
def Distance_finder(Focal_Length, Known_width, obj_width_in_frame):
    distance = (Known_width * Focal_Length) / obj_width_in_frame
    return distance

# Load a reference image and extract the object width from it
ref_image = cv2.imread("rf.png")
ref_image_obj_width = obj_data(ref_image)
# Calculate focal length based on the reference image
Focal_length_found = Focal_Length_Finder(Known_distance, Known_width, ref_image_obj_width)
cv2.imshow("ref_image", ref_image)

# Placeholder for an "arm" function, which presumably controls a drone
def arm():
    pass

while True:
    ret, frame = cap.read()
    frame = cv2.resize(frame, (640, 480))
    obj_width_in_frame = obj_data(frame)

    # If an object is detected, calculate its distance
    if obj_width_in_frame != 0:
        Object_distance = Distance_finder(Focal_length_found, Known_width, obj_width_in_frame)
        Detected_object = 1
    else:
        Detected_object = 0
        Object_distance = 0

    # Display information about the detected object and its distance
    cv2.putText(frame, f"Distance: {round(Object_distance, 2)} CM", (30, 35), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 0, 0), 2)
    cv2.imshow("FRAME", frame)

    # If the detected object is within a specific distance range, stop drone movement and call the "arm" function
    if (Detected_object == 1) and (25 < round(Object_distance, 2) < 30):
        # Stop drone movement
        # Call arm function
        arm()
        print(f"Distance: {round(Object_distance, 2)} CM")

    # Break the loop if the 'Esc' key is pressed
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()
